# Game Setup Testing

In [1]:
from MTG_Engine_v5 import *

In [ ]:
class GameState:
    def __init__(self, players: List[Player]):
        self.players = players
        self.num_players = len(players)
        self.active_player_index = 0
        self.current_phase: Optional[TurnPhase] = None
        self.current_step: Optional[TurnStep] = None
        self.stack = Stack()
        self.consecutive_passes = 0
        self.player_with_priority_index = 0
        self.sba_queue: List[SBA] = []
        self.event_manager = EventManager(self)
        self.pending_triggers: List[TriggeredAbility] = []
        self.current_turn = 0

        self.replacement_effects: Dict[Event, ReplacementEffect] = {}
        self.seen_effects: List[ReplacementEffect] = []

        self.trigger_listeners = Dict[Event, TriggeredAbility] = {}
        self.active_modifiers: List[Modifier] = []

    def setup_game(self):
        print("\n--- Setting up game ---")
        
        # 1. Randomly determine turn order
        print("Randomizing turn order...")
        self._set_turn_order()


        # 2. Players shuffle their libraries and draw opening hands
        self._loop_priority([ [Player.shuffle_library], [Player.draw_cards, {'num': 7}] ])
            

        # 3. Handle the mulligan process
        self._loop_priority([ [Player._prompt_mulligan, {}] ])

        # 4. Handle pre-game actions
        # placeholder for now
            
        print("\n--- Game setup complete ---\n")

    def game_loop(self):
        while True:
            self.current_turn += 1
            turn_num = self.current_turn

            print(f"\n--- Turn {turn_num} ---\n")

            self._loop_priority([ [self._take_turn, {}]  ])
            

            break # Placeholder ------------------------

    def _take_turn(self, player: Player):
        if not player.alive:
            return
        
        phases = [TurnPhase.BEGINNING, TurnPhase.PRECOMBAT_MAIN, TurnPhase.COMBAT, TurnPhase.POSTCOMBAT_MAIN, TurnPhase.ENDING] # Making this a list to make it more flexible for things such as adding more phases
        steps = {
            TurnPhase.BEGINNING: [TurnStep.UNTAP, TurnStep.UPKEEP, TurnStep.DRAW],
            TurnPhase.PRECOMBAT_MAIN: [TurnStep.MAIN],
            TurnPhase.COMBAT: [TurnStep.BEGINNING_OF_COMBAT, TurnStep.DECLARE_ATTACKERS, TurnStep.DECLARE_BLOCKERS, TurnStep.FIRST_STRIKE_DAMAGE, TurnStep.COMBAT_DAMAGE, TurnStep.END_OF_COMBAT],
            TurnPhase.POSTCOMBAT_MAIN: [TurnStep.MAIN], 
            TurnPhase.ENDING: [TurnStep.END, TurnStep.CLEANUP]
        }

        phase_num = 0
        while phase_num < len(phases):
            self.current_phase = phases[phase_num] # Set phase
            step_num = 0

            # 1. Advance step
            while step_num < len(steps[self.current_phase]):
                self.current_step = steps[self.current_phase][step_num] # Set step

                # 1.5 Check for skip effects
                ############

                # 2. Turn-Based Actions
                self._turn_based_actions(player)

                # 3. Check for begginning of step triggers
                # Post begginning of step trigger

                # 4. Players receive priority
                #If we are in untap, first strike, combat damage, or cleanup, no priority is received
                if self.current_step not in [TurnStep.UNTAP, TurnStep.FIRST_STRIKE_DAMAGE, TurnStep.COMBAT_DAMAGE, TurnStep.CLEANUP]:
                    #self._loop_priority([[]]) --- get decision based on legal actions
                    pass

                # 5. Check for end of step triggers
                # Post end of step trigger

                # Go to next step
                step_num += 1
            
            # Go to next phase
            phase_num += 1
            
    def _turn_based_actions(self, player):
        step = self.current_step

        match step: # These are the only steps where TBAs take place
            case TurnStep.UNTAP:
                # modify this to post an event to possibly replace the untapping with something (i.e., winter orb)
                self.untap(player.battlefield)
            case TurnStep.DRAW:
                player.draw_cards(1) # Handle triggers and replacement here
            case TurnStep.DECLARE_ATTACKERS:
                # Active player declares attackers
                pass
            case TurnStep.DECLARE_BLOCKERS:
                # Players being attacked declare blockers
                pass
            case TurnStep.FIRST_STRIKE_DAMAGE:
                # Deal first strike damage
                pass
            case TurnStep.COMBAT_DAMAGE:
                # Deal combat damage
                pass
            case TurnStep.CLEANUP:
                if len(player.hand) > player.max_hand_size:
                    # Prompt player to choose cards to discard
                    pass
                # Remove all damage marked on permanents
                # End until end of turn effects

    def untap(self, permanents: List['Permanent'], types: Set[str] = []): # Need to revamp this to make restrictions more intuitive
        for permanent in permanents:
            # Check if we can untap
            if (self.current_step == TurnStep.UNTAP and permanent.untaps_in_untap and permanent.untappable) or (self.current_step != TurnStep.UNTAP and permanent.untappable):
                if types and condition_check(permanent, CheckCondition.TYPE, types):
                    self.EventManager(permanent, Permanent.untap, Event.UNTAP)

    def EventManager(self, object: Any, action: Callable, event: Event, **params): # Anything that is going to change our GameState should run through here
        # First check if replacement effects exist and replace our event if necessary
        replacement_effects = self.replacement_effects[event]
        replacement_types = [replacement_effect.replacementType for replacement_effect in replacement_effects]

        # Cancel if a skip replacement is present
        if ReplacementType.SKIP in replacement_types: 
            return 

        # Note self.replacement_effects: Dict[Event, List[ReplacementEffect]]
        for replacement_effect in replacement_effects:
            if replacement_effect in self.seen or not self._check_replacement_conditions(event, replacement_effect.replacementConditions, params):
                continue

            # Placeholder -- # Get info from player on which effect to perform first that hasn't been done
            self.seen.append(replacement_effect)

            action, params, event = self._execute_replacements(action, replacement_effect, params)
            return self.EventManager(object, action, event, params)
        
        # Then check if any trigger listens exist and add triggers to the stack
        if event in self.trigger_listeners:
            for triggered_ability in self.trigger_listeners[event]:
                triggered_ability.trigger(params)

                # Finally, execute event
                action(object, params)
                self.seen_effects.clear() # Reset seen

    def _execute_replacements(self, action, replacement_effect: ReplacementEffect, **params):
        event = replacement_effect.event

        # Execute replacements
        match replacement_effect.replacementType:
            case ReplacementType.QUANTITY_MODIFICATION:
                params['num'] = replacement_effect.modification(params['num'])
            case ReplacementType.ZONE_REDIRECT:
                params['new_zone'] = replacement_effect.modification
            case ReplacementType.ACTION_CHANGE:
                action = replacement_effect.modification
                event = replacement_effect.new_event if replacement_effect.new_event else event 

        return action, params, event

    def _check_replacement_conditions(self, event: Event, replacement_condition: tuple[Comparator, Dict], **params):
        comparator = replacement_condition[0].value
        replacement_params = replacement_condition[1]
        
        match event:
            case Event.ZONE_CHANGE:
                if replacement_params['old_zone']:
                    return comparator(replacement_params['old_zone'], params['card'].zone) and comparator(replacement_params['new_zone'], params['new_zone'])
            case Event.DEAL_DAMAGE:
                return comparator(replacement_params['num'], params['num']) and comparator(replacement_params['damage_type'], params['damage_type'])
            case _:
                return comparator(replacement_params['num'], params['num'])

    def _register_replacement_effect(self, replacement_effect: ReplacementEffect):
        self.replacement_effects.get(replacement_effect.event,[]).append(replacement_effect)

    def _deregister_replacement_effect(self, replacement_effect: ReplacementEffect):
        self.replacement_effects.get(replacement_effect.event,[]).remove(replacement_effect)

    def _register_trigger_listener(self, event: Event, triggered_ability: Callable):
        self.trigger_listeners.get(event, []).append(triggered_ability)

    def _deregister_trigger_listener(self, event: Event, triggered_ability: Callable):
        self.trigger_listeners.get(event, []).remove(triggered_ability)

    def _insert_step(self, steps: List[TurnStep], StepType: TurnStep, index: int):
        steps.insert(index + 1, StepType)
    
    def _insert_phase(self, phases: List[TurnPhase], PhaseType: TurnPhase, index: int):
        phases.insert(index + 1, PhaseType)

    def _set_turn_order(self):
        random.shuffle(self.players)
        print(f"Turn order: {' - '.join(player.name for player in self.players)}")
    
    def _loop_priority(self, actions: List[List[Any]]):
        active_player = self.active_player_index
        num_players = self.num_players

        for i in range(active_player, active_player + num_players):
            current_player = self.players[i % num_players] # Loop through each player starting with active player
            for item in actions: # Loop through each method / action we want them to take
                action = item[0]
                if len(item) > 1:
                    params = item[1]
                else:
                    params = {}

                action(current_player, **params) # Execute
                        
    def create_permanent_from_card(self, card: Card, controller: 'Player') -> Permanent:
        new_permanent = Permanent(card, self)
        new_permanent.owner = card.owner
        new_permanent.controller = controller
        controller.battlefield.append(new_permanent)
        print(f"    -> {new_permanent.name} enters the battlefield under {controller.name}'s control.")
        return new_permanent
    
    def destroy_permanent(self, permanent: Permanent):
        permanent.controller.battlefield.remove(permanent)

    def get_legal_action_types(self, player: 'Player') -> List[ActionType]:
        # Placeholder for full legal action checking
        action_types = [ActionType.PASS_PRIORITY]
        # A full implementation would check timing, permissions, etc.
        if any("Land" not in c.types for c in player.hand):
            action_types.insert(0, ActionType.CAST_SPELL)
        return action_types # Note that action_types doesn't return every possible action, but just the possible actions that the player can take

    def validate_action(self, action: Action) -> bool:
        player = action.player
        if isinstance(action, CastSpellAction):
            card = action.data['card']
            if card not in player.hand: return False
            # The can_pay_cost check here is using the placeholder method.
            if not player.can_pay_cost({}): return False
            # A full implementation would also check for timing permission (sorcery vs. instant speed).
        return True

    def _check_state_based_actions(self):
        """
        Continuously processes the SBA queue until the game state is stable.
        This is the core of the SBA engine.
        """
        while True:
            # Move the current queue to a processing list and clear the main queue.
            # This handles cases where resolving an SBA queues up another one.
            if not self.sba_queue:
                break # No checks are pending, so the state is stable.

            checks_to_process = list(self.sba_queue)
            self.sba_queue.clear()
            
            actions_taken = False
            for check in checks_to_process:
                if check.type == SBAType.CHECK_PLAYER_LIFE:
                    player = check.data['player']
                    if player.life <= 0:
                        print(f"  [SBA] {player.name} has {player.life} life and loses the game.")
                        # Future implementation: move player to a 'lost' state.
                        actions_taken = True

                elif check.type == SBAType.CHECK_LETHAL_DAMAGE:
                    permanent = check.data['permanent']
                    # Ensure the permanent is still on the battlefield before checking.
                    if permanent in permanent.controller.battlefield and "Creature" in permanent.types:
                        if permanent.damage_marked >= permanent.toughness:
                            print(f"  [SBA] {permanent.name} has lethal damage ({permanent.damage_marked}/{permanent.toughness}).")
                            self.destroy_permanent(permanent)
                            actions_taken = True
                
                elif check.type == SBAType.CHECK_COMMANDER_DAMAGE:
                    player = check.data['player']
                    for commander_uuid, damage in player.commander_damage.items():
                        if damage >= 21:
                            print(f"  [SBA] {player.name} has taken {damage} commander damage and loses the game.")
                            # Future implementation: move player to a 'lost' state.
                            actions_taken = True
                            break # Player has already lost, no need to check other commanders.
                
                elif check.type == SBAType.CHECK_LEGEND_RULE:
                    permanent = check.data['permanent']
                    for perm in permanent.controller.battlefield:
                        if perm.name == permanent.name:
                            keep_choice = InputHandler.get_decision(permanent.controller, "Choose which creature to keep", [perm, permanent])
                            if keep_choice == permanent:
                                permanent.d
            
            # If a full pass resulted in no actions, the state is stable.
            if not actions_taken:
                break

    def _stack_pending_triggers(self):
        if not self.pending_triggers: return
        ap_index = self.active_player_index
        for i in range(len(self.players)):
            player_index = (ap_index + i) % len(self.players)
            player = self.players[player_index]
            player_triggers = [t for t in self.pending_triggers if t.controller == player]
            if not player_triggers: continue
            
            ordered_triggers = player_triggers
            if len(player_triggers) > 1:
                ordered_triggers = InputHandler.get_player_choice_for_trigger_order(player, player_triggers)
            
            for trigger in ordered_triggers:
                self.stack.add(trigger, player)
                self.pending_triggers.remove(trigger)
        
        self.pending_triggers.clear()


IndentationError: expected an indented block after function definition on line 134 (2567334213.py, line 140)

Load in players and start pregame

In [ ]:
# Load decks
deck1 = Deck(Card({}),[]) # dummy for now

# Set players
player1 = Player("p1", deck1)
player2 = Player("p2", deck1)
player3 = Player("p3", deck1)
player4 = Player("p4", deck1)
players = [player1, player2, player3, player4]

# Load a gamestate
engine = GameState(players)

# Set up game
engine.setup_game()

# Start game loop
engine.game_loop()